# 3D Toric Code Sweep Decoder

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pymatching import Matching
import os
from bn3d.bpauli import get_effective_error
from bn3d.app import (
    read_input_dict
)
from bn3d.io import (
    serialize_results, dump_results
)
from tqdm.notebook import tqdm
from bn3d.plots import plot_export_json
import IPython.display
import datetime
from bn3d.config import BN3D_DIR, BN3D_DARK_THEME
if BN3D_DARK_THEME:
    plt.style.use('dark_background')

import pandas as pd

In [ ]:
# Parameters
# Change these as you wish.

input_data = {
    'comments': 'Simple Demonstration',
    'ranges': {
        'label': 'demo_01',
        'code': {
            'model': 'ToricCode3D',
            'parameters': [
                {'L_x': 4},
                {'L_x': 6},
                {'L_x': 8},
            ]
        },
        'noise': {
            'model': 'PauliErrorModel',
            'parameters': [
                {'r_x': 0, 'r_y': 0, 'r_z': 1}
            ],
        },
        'decoder': {
            'model': 'SweepMatchDecoder'
        },
        'probability': np.arange(0.1, 0.2, 0.01).round(5).tolist(),
    }
}

# Frequecy of plot update.
plot_frequency = 5

# Frequency of saving to file.
save_frequency = 20

# Target number of simulations.
n_trials = 1000

In [ ]:
batch_sim = read_input_dict(
    input_data,
    update_frequency=plot_frequency,
    save_frequency=save_frequency
)

In [ ]:
def update_plot():
    """Plot routine on loop."""
    remaining_time = batch_sim.estimate_remaining_time(n_trials)
    df = pd.DataFrame(batch_sim.get_results())

    plt.clf()
    for code_size in df['size'].unique():
        df_filtered = df[df['size'] == code_size]
        plt.errorbar(
            df_filtered['probability'], df_filtered['p_est'],
            yerr=df_filtered['p_se'],
            label=f'L={code_size[0]}'
        )
    plt.title(f'Time remaining {datetime.timedelta(seconds=int(remaining_time))}')
    plt.xlabel('Physical Error Rate', fontsize=20)
    plt.ylabel('Logical Error Rate', fontsize=20)
    plt.legend()
    IPython.display.clear_output(wait=True)
    IPython.display.display(plt.gcf())

batch_sim.on_update = update_plot

In [ ]:
batch_sim._run(n_trials, progress=tqdm)